In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## for feature slection

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

### Before model building we have to do Feature selection and remove highly correlated features

In [4]:
train = pd.read_csv("train_data_scaled.csv")
train.head()

,id,churn,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,origin_up,pow_max,forecast_discount_energy_nan,forecast_price_energy_p1_nan,forecast_price_energy_p2_nan,forecast_price_pow_p1_nan,margin_gross_pow_ele_nan,margin_net_pow_ele_nan,net_margin_nan,pow_max_nan,Length Of Contract (Year)
0,48ada52261e7cf58715202705a0451c9,0,1.0,-1.000000,1.413248,0.000000,0.289269,1.991987,0.0,1.406780,-1.530329,0.022097,3.963793,0.0,0.539450,0.000000,0.000000,0.0,0.378568,0.0,0.622746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333
1,24011ae4ebbe3035111d65fa7c15bc57,1,0.0,0.333333,0.132764,4.739944,-0.817317,0.000000,0.0,-0.978104,-0.924658,0.121200,-1.000000,1.0,-0.708312,0.313853,0.322511,1.0,2.658602,0.5,4.081096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.666667
2,d29c2c54acc38ff3c0614d0a653813dd,0,0.0,-0.666667,-0.591831,0.000000,-0.817317,-1.198750,0.0,-0.082621,0.094365,-0.871769,0.000000,0.0,-0.708312,-0.339827,-0.331169,0.0,-0.479091,-0.5,-0.007671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667
3,764c75f661154dac3a6c254cd082ea7d,0,0.0,0.333333,-1.617646,0.000000,-0.817317,-2.078946,0.0,0.325714,0.749666,0.017564,0.000000,0.0,-0.708312,0.541847,0.550505,0.0,-0.537510,-0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
4,bba03439a292a1e166f80264c16191cb,0,0.0,-1.000000,-1.107389,0.000000,-0.817317,-1.048095,0.0,0.009266,0.126440,-0.871769,0.000000,0.0,-0.708312,0.658730,0.667388,0.0,-0.447862,-0.5,-0.089863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333


In [5]:
y_train = train["churn"]
x_train = train.drop(['id', 'churn'], axis=1)

In [21]:
x_train

,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,origin_up,pow_max,forecast_discount_energy_nan,forecast_price_energy_p1_nan,forecast_price_energy_p2_nan,forecast_price_pow_p1_nan,margin_gross_pow_ele_nan,margin_net_pow_ele_nan,net_margin_nan,pow_max_nan,Length Of Contract (Year)
0,1.0,-1.000000,1.413248,0.000000,0.289269,1.991987,0.0,1.406780,-1.530329,0.022097,3.963793,0.0,0.539450,0.000000,0.000000,0.0,0.378568,0.0,0.622746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333
1,0.0,0.333333,0.132764,4.739944,-0.817317,0.000000,0.0,-0.978104,-0.924658,0.121200,-1.000000,1.0,-0.708312,0.313853,0.322511,1.0,2.658602,0.5,4.081096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.666667
2,0.0,-0.666667,-0.591831,0.000000,-0.817317,-1.198750,0.0,-0.082621,0.094365,-0.871769,0.000000,0.0,-0.708312,-0.339827,-0.331169,0.0,-0.479091,-0.5,-0.007671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667
3,0.0,0.333333,-1.617646,0.000000,-0.817317,-2.078946,0.0,0.325714,0.749666,0.017564,0.000000,0.0,-0.708312,0.541847,0.550505,0.0,-0.537510,-0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
4,0.0,-1.000000,-1.107389,0.000000,-0.817317,-1.048095,0.0,0.009266,0.126440,-0.871769,0.000000,0.0,-0.708312,0.658730,0.667388,0.0,-0.447862,-0.5,-0.089863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,0.0,0.333333,0.333009,4.680707,-0.817317,0.865850,0.0,-0.021325,-0.147290,-0.871769,0.000000,1.0,-0.708312,0.489899,0.498557,1.0,1.245808,0.5,0.156712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333
16092,0.0,0.333333,-0.382395,0.000000,-0.192253,-0.423995,0.0,0.960652,-1.391719,0.057964,3.963793,0.0,-0.183275,-1.521645,-1.512987,0.0,-0.139464,0.5,-1.076164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333
16093,0.0,0.333333,-1.034788,0.000000,-0.193580,-1.197261,0.0,0.909272,-0.845727,0.140150,-1.000000,0.0,-0.161494,1.352814,1.361472,0.0,-0.472009,0.5,0.284795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333
16094,0.0,0.333333,-2.295398,0.000000,-0.817317,-2.647038,0.0,-0.448987,0.094365,-0.871769,0.000000,0.0,-0.708312,-0.577922,-0.569264,0.0,-0.564319,0.5,-0.391233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333


Before building the model we will check the multicollinearity among the features with variance inflation factor

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_train)

In [36]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variables = x_scaled
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables,i) for i in range(variables.shape[1])]
vif["Features"]=x_train.columns

C:\Users\DELL\Anaconda\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [30]:
to_drop = vif[vif["VIF"]>5].Features.values

In [31]:
x_train_new = x_train.drop(to_drop, axis=1)

In [33]:
from sklearn.preprocessing import StandardScaler
scaler_new = StandardScaler()
x_scaled_new = scaler.fit_transform(x_train_new)

In [34]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variables = x_scaled_new
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables,i) for i in range(variables.shape[1])]
vif["Features"]=x_train_new.columns

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x_train_new, y_train, test_size=0.25,random_state=255)

### Model Building  
As this is a classification approach we will use two best known classifiers RandomForests and XG Boost. And then we will fine tune them

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

In [43]:
rand_clf = RandomForestClassifier(random_state=6)
rand_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=6)

In [44]:
rand_clf.score(x_test,y_test)

0.9003479125248509

In [46]:
hyperparameters = {
    "n_estimators" : [90,100,115,130],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}

In [48]:
randomized_search = RandomizedSearchCV(rand_clf, hyperparameters, random_state=1, n_iter=100,cv=5,n_jobs=-1,verbose=0)

In [49]:
randomized_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=6),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(2, 20),
                                        'max_features': ['auto', 'log2'],
                                        'min_samples_leaf': range(1, 10),
                                        'min_samples_split': range(2, 10),
                                        'n_estimators': [90, 100, 115, 130]},
                   random_state=1)

In [50]:
randomized_search.best_params_

{'n_estimators': 100,
 'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 18,
 'criterion': 'gini'}

In [53]:
rand_clf = RandomForestClassifier(criterion= 'gini',
 max_features = 'log2',
 min_samples_leaf = 1,
 min_samples_split= 8,
 n_estimators = 100,max_depth=18,random_state=6)

rand_clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=18, max_features='log2', min_samples_split=8,
                       random_state=6)

In [54]:
rand_clf.score(x_test, y_test)

0.8998508946322068

Even if the score of the model has decreased, but the model is mode stable the before.   
Hyperparameter tuning does not improve the accuracy, it increases the stability of the model.

### Using XGBoost Classifier

In [57]:
from xgboost import XGBClassifier

In [59]:
xgb_model = XGBClassifier(objective="binary:logistic")
xgb_model.fit(x_train, y_train)

C:\Users\DELL\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:19:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [61]:
xgb_model.score(x_test, y_test)

0.8996023856858847

In [67]:
hyperparameter_xgb={
    'learning_rate':[1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
}

In [68]:
randomized_search_xgb = RandomizedSearchCV(xgb_model, hyperparameter_xgb, random_state=1, n_iter=100,cv=5,n_jobs=-1,verbose=3)

In [69]:
randomized_search_xgb.fit(x_train, y_train)

C:\Users\DELL\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 80 is smaller than n_iter=100. Running 80 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 80 candidates, totalling 400 fits


C:\Users\DELL\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:30:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_delta_step=0, max_depth=6,
                                           min_child_weight=1, missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=100, n_jobs=4,
                                           num_parallel_tree=1, random_state=0,
                                           reg_alpha=0, reg_lambda=1,
                                       

In [70]:
randomized_search_xgb.best_params_

{'n_estimators': 50, 'max_depth': 20, 'learning_rate': 0.1}

In [73]:
new_model=XGBClassifier(learning_rate= 0.1, max_depth= 20, n_estimators= 50)
new_model.fit(x_train, y_train)

[09:32:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [74]:
new_model.score(x_test, y_test)

0.9005964214711729

We achieved 90% accuracy with XGB Classifier which is better than RandomForests and also our model is not overfitted, so we will use XGB Classifier

Save the model

In [75]:
import pickle

In [76]:
filename = "xgb_model.pickle"
pickle.dump(new_model, open(filename, "wb"))